#Athena Client Setup

In [35]:
import boto3
from google.colab import userdata

AWS_ACCESS_KEY = userdata.get('aws_access_key_id')
AWS_SECRET_KEY = userdata.get('aws_secret_access_key')
AWS_REGION = "us-east-2"
output_bucket = 's3://click-event-analysis/query-results/'

athena_client = boto3.client(
    "athena",
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=AWS_REGION,
)

In [47]:
import time
import pandas as pd


def execute_query(query):
    response = athena_client.start_query_execution(
      QueryString=query,
      QueryExecutionContext={
          'Database': 'click_event_analysis-db',
          'Catalog': 'AwsDataCatalog'
      },
      ResultConfiguration={'OutputLocation': output_bucket}
    )

    query_execution_id = response['QueryExecutionId']


    while True:
        response = athena_client.get_query_execution(QueryExecutionId=query_execution_id)
        status = response['QueryExecution']['Status']['State']
        if status == 'SUCCEEDED':
            break
        elif status == 'FAILED':
            raise Exception("Query failed: " + response['QueryExecution']['Status']['StateChangeReason'])
        time.sleep(1)


    result = athena_client.get_query_results(QueryExecutionId=query_execution_id)

    rows = result['ResultSet']['Rows']
    header = [col['VarCharValue'] for col in rows[0]['Data']]
    data = [[col.get('VarCharValue', None) for col in row['Data']] for row in rows[1:]]

    return pd.DataFrame(data, columns=header)

#Most Clicked Category by Location

In [ ]:
query = '''
      SELECT
        category,
        location,
        SUM(click_count) as total_clicks
      FROM
        category_clicks_by_location
      GROUP BY
        category,
        location
      '''
df_mccbl = execute_query(query)
print(df_mccbl)

In [63]:
import plotly.express as px

heatmap_data = df_mccbl.pivot(index="location", columns="category", values="total_clicks")

fig = px.imshow(
    heatmap_data,
    labels=dict(x="Category", y="Location", color="Total Clicks"),
    x=heatmap_data.columns,
    y=heatmap_data.index,
    title="Most Clicked Category by Location",
    color_continuous_scale="RdYlGn_r"
)


fig.update_layout(
    xaxis_nticks=len(heatmap_data.columns),
    width=600,
    height=600
)

fig.show()

In [67]:
query = """
SELECT
    location,
    SUM(click_count) AS total_clicks
FROM
    category_clicks_by_location
GROUP BY
    location
ORDER BY
    total_clicks DESC
"""

df_lwmc = execute_query(query)
print(df_lwmc)

In [71]:
import plotly.express as px

fig = px.pie(df_lwmc,
             names="location",
             values="total_clicks",
             title="Locations With Most Clicks")

fig.update_layout(
    width=600,
    height=600
)

fig.show()


Category

#Most Clicked Category by Platform

In [ ]:
query = '''
      SELECT
        category,
        platform,
        SUM(click_count) as total_clicks
      FROM
        category_clicks_by_platform
      GROUP BY
        category,
        platform
      '''
df_mccbp = execute_query(query)
print(df_mccbp)

In [66]:
import plotly.express as px

heatmap_data = df_mccbp.pivot(index="platform", columns="category", values="total_clicks")

fig = px.imshow(
    heatmap_data,
    labels=dict(x="Category", y="Platform", color="Total Clicks"),
    x=heatmap_data.columns,
    y=heatmap_data.index,
    title="Most Clicked Category by Platform",
    color_continuous_scale="RdYlGn_r"
)


fig.update_layout(
    xaxis_nticks=len(heatmap_data.columns),
    width=600,
    height=600
)

fig.show()

In [ ]:
query = """
SELECT
    platform,
    SUM(click_count) AS total_clicks
FROM
    category_clicks_by_platform
GROUP BY
    platform
ORDER BY
    total_clicks DESC
"""

df_pwmc = execute_query(query)
print(df_pwmc)

In [77]:
import plotly.express as px

fig = px.pie(df_pwmc,
             names="platform",
             values="total_clicks",
             title="Platforms With Most Clicks")

fig.update_layout(
    width=600,
    height=600
)

fig.show()
